<a href="https://colab.research.google.com/github/WellcomePeujio/Analisis-de-Impacto-del-PIB-en-los-Mercados-Financieros/blob/main/An%C3%A1lisis_de_Impacto_del_PIB_en_los_Mercados_Financieros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Impacto del PIB en los Mercados Financieros

## **Título sugerido**: "¿El crecimiento económico impulsa las inversiones?"

### **Problema**  
Contamos con un conjunto de datos que reúne el **crecimiento trimestral del PIB de Estados Unidos** y el **rendimiento del índice S&P 500** durante los últimos **20 años**.

Queremos analizar si existe una relación estadísticamente significativa entre el comportamiento de la economía real (PIB) y los rendimientos bursátiles (S&P 500).

---

### **Preguntas a resolver**

1️⃣ Ajusta un **modelo econométrico** para estimar la relación entre el **crecimiento del PIB** y el **rendimiento del S&P 500**.

2️⃣ Utiliza el modelo ajustado para **predecir el rendimiento del S&P 500 en caso de una recesión**, es decir, cuando el crecimiento del PIB es negativo.

3️⃣ Genera una **visualización con un gráfico de dispersión**, donde se muestre la correlación entre el **crecimiento del PIB** y el **rendimiento del S&P 500**, incluyendo una **línea de regresión lineal**.

---

### **Fórmula general del modelo econométrico**

$$
R_{S\&P500} = \alpha + \beta \cdot \Delta PIB + \epsilon
$$

Donde:
- $R_{S\&P500}$ es el rendimiento del índice S&P 500 en un trimestre.
- $\Delta PIB$ es la tasa de crecimiento del Producto Interno Bruto.
- $\alpha$ es la constante del modelo.
- $\beta$ es el coeficiente de sensibilidad del S&P 500 al crecimiento del PIB.
- $\epsilon$ es el término de error aleatorio.

---


# Resolución Matemática: Análisis de Impacto del PIB en los Mercados Financieros

## Paso 1: Modelo Econométrico

El objetivo es estimar la relación entre el **crecimiento del PIB** y el **rendimiento del S&P 500** mediante un modelo de regresión lineal simple:

$$
R_{S\&P500} = \alpha + \beta \cdot \Delta PIB + \epsilon
$$

Donde:
- $R_{S\&P500}$ es el rendimiento del índice en un trimestre.
- $\Delta PIB$ es el crecimiento trimestral del PIB.
- $\alpha$ es el intercepto del modelo.
- $\beta$ es el coeficiente que mide el impacto del PIB sobre el mercado.
- $\epsilon$ es el error aleatorio del modelo.

---

## Paso 2: Hipótesis del Modelo

Para aplicar este modelo, planteamos las siguientes hipótesis:

- $H_0$: $\beta = 0$ → No hay relación entre el crecimiento del PIB y el S&P 500.
- $H_1$: $\beta \neq 0$ → Existe una relación significativa.

Si $\beta$ resulta **significativamente distinto de cero**, entonces podemos afirmar que el crecimiento económico tiene impacto en el rendimiento bursátil.

---

## Paso 3: Ajuste del Modelo con Datos Reales

Dado un conjunto de observaciones trimestrales $\{ (\Delta PIB_t, R_t) \}$ para $t = 1, 2, ..., N$, estimamos los coeficientes $\alpha$ y $\beta$ utilizando el método de **mínimos cuadrados ordinarios (OLS)**:

$$
\hat{\beta} = \frac{\sum_{t=1}^N (\Delta PIB_t - \overline{\Delta PIB})(R_t - \overline{R})}{\sum_{t=1}^N (\Delta PIB_t - \overline{\Delta PIB})^2}
$$

$$
\hat{\alpha} = \overline{R} - \hat{\beta} \cdot \overline{\Delta PIB}
$$

---

## Paso 4: Predicción ante Recesión

Para simular el comportamiento del mercado en caso de **recesión** (por ejemplo, $\Delta PIB = -1\%$), usamos el modelo estimado:

$$
\hat{R}_{S\&P500} = \hat{\alpha} + \hat{\beta} \cdot (-0.01)
$$

Este resultado nos permite predecir la **respuesta del mercado** ante un retroceso económico.

---

## Paso 5: Visualización y Análisis

Se recomienda representar los datos con un **gráfico de dispersión**, donde:

- El eje $x$ representa $\Delta PIB$
- El eje $y$ representa $R_{S\&P500}$
- Se incluye la **línea de regresión** que representa el modelo ajustado.

Esta visualización permite observar la tendencia, correlación y posibles anomalías entre variables.

---

## Conclusión

Este análisis permite responder empíricamente a la pregunta:  
**¿El crecimiento económico impulsa las inversiones?**  
Si el coeficiente $\hat{\beta}$ es positivo y significativo, se valida la hipótesis de que **el PIB sí influye positivamente en el rendimiento bursátil**.

---


In [ ]:
import pandas as pd
import yfinance as yf
from pandas_datareader.data import DataReader
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Fechas para los últimos 20 años
start = datetime.datetime(2003, 1, 1)
end = datetime.datetime(2023, 12, 31)

# Descargar datos del PIB real trimestral desde FRED
gdp = DataReader('GDPC1', 'fred', start, end)
gdp = gdp.resample('Q').last()
gdp['GDP_growth'] = gdp['GDPC1'].pct_change()  # Crecimiento trimestral del PIB

# Descargar precios del S&P 500 (mensual, luego se resamplea a trimestral)
sp500 = yf.download('^GSPC', start=start, end=end, interval='1mo')['Close']
sp500 = sp500.resample('Q').last()
sp500_returns = sp500.pct_change()  # Rendimiento trimestral

# Combinar los datos en un solo DataFrame
df = pd.concat([gdp['GDP_growth'], sp500_returns], axis=1)
df.columns = ['GDP_growth', 'SP500_return']
df = df.dropna()

# Modelo de regresión lineal simple
X = sm.add_constant(df['GDP_growth'])
y = df['SP500_return']
model = sm.OLS(y, X).fit()

# Predicción si el crecimiento del PIB es -1%
predicted_return = model.predict([1, -0.01])[0]

# Gráfico de dispersión con línea de regresión
plt.figure(figsize=(10, 6))
plt.scatter(df['GDP_growth'], df['SP500_return'], alpha=0.6, label='Datos históricos')
plt.plot(df['GDP_growth'], model.predict(X), color='red', label='Regresión lineal')
plt.axvline(-0.01, color='gray', linestyle='--', label='Recesión (-1% PIB)')
plt.axhline(predicted_return, color='green', linestyle='--', label=f'Predicción S&P 500: {predicted_return:.2%}')
plt.xlabel('Crecimiento del PIB (trimestral)')
plt.ylabel('Rendimiento del S&P 500 (trimestral)')
plt.title('Relación entre PIB y S&P 500 (2003-2023)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Mostrar resultados del modelo
model.summary(), predicted_return